In [1]:
import pandas as pd

In [2]:
df = pd.read_fwf("signal1m_tweets_qrels", names=['topic','iteration','document','rel'])

df[df.document==645936801232912384]
df.shape

(6103, 4)

In [3]:
import tweepy as tw
'''
if need to log in to the twitter account:
username = eugene_tan@mymail.sutd.edu.sg
password = P@ssword1
'''

consumer_key= 'LKvavXktNliE3LL52KNK48oWd'
consumer_secret= 'PsSoEz5JRYdrozOe8uBkgphm3YlTipYREexIyJQi4VDVZv9htI'
access_token= '1411277462630146057-ATYdgNSGb3EGBFim8w1sQ8kRGOR7QH'
access_token_secret= 'S3N1R58XNvrucIOgsKIxZY4CSfngnrd6XHIlWCu0k4PaP'
bearer_token= 'AAAAAAAAAAAAAAAAAAAAAOskRQEAAAAA5JKli8luYWK%2BzRTWfn1kLwjv58s%3DWNBJ8KrY0xEwZBqxqLzCjde2fGWM3wBMeq4uyvWoLX5BNfIHHp'

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [4]:
## Scrape Data

# def get_tweet(id):
#     output = ''
#     try:
#         output = api.get_status(id=id.document).text
#     except:
#         pass
#     return output

# df['text']=df.apply(lambda x:get_tweet(x), axis=1)

# df.head()
# df = df[df.text!='']
# df.to_csv("dataset_scrapped.csv", header=False)

In [19]:
data = pd.read_csv('dataset_scrapped.csv', names=['topic','iteration','document','rel', 'text'])
data.head()

,topic,iteration,document,rel,text
0,00ecb565-8dcb-443e-875c-71babdce2269,0,638637447023194112,0,Video: Shippers look to grab a piece of North...
1,00ecb565-8dcb-443e-875c-71babdce2269,0,638727826393444352,1,Police say woman was driving drunk with kids i...
2,00ecb565-8dcb-443e-875c-71babdce2269,0,639043756038074369,1,Hit-and-Run Driver Seriously Injures Man in Po...
3,00ecb565-8dcb-443e-875c-71babdce2269,0,639219133859631104,0,#UPDATE: Man in critical condition after being...
4,00ecb565-8dcb-443e-875c-71babdce2269,0,639325701297041408,1,A Barrie woman that was run over by a drunk dr...


In [31]:
from nltk.corpus import words
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import nltk

data = pd.read_csv('dataset_scrapped.csv', names=['topic','iteration','document','rel', 'text'])

porter = PorterStemmer() ## stemming reduces the word to root word
words = words.words()
# vsm = {}

# def stem_lower(word):
#     return porter.stem(word.lower())

# for i in list(set(zip(data['document'], data['text']))):
#     # print(i[0], i[1])
#     tokens = word_tokenize(i[1])
#     # print(len(tokens))
#     tokens = list(filter(lambda x: x.isalpha(), tokens))
#     # tokens = list(filter(lambda x: x in words, tokens)) ## this removes too much semantics
#     tokens = list(map(stem_lower, tokens))
#     # print(len(tokens))
#     vsm[str(i[0])] = tokens

def tokenize_stem_lower(text):
    tokens = word_tokenize(text)
    tokens = list(filter(lambda x: x.isalpha(), tokens))
    tokens = [porter.stem(x.lower()) for x in tokens]
    return ' '.join(tokens)

data['text']=data.apply(lambda x: tokenize_stem_lower(x.text), axis=1)

vsm['645687105964912640']
data.head(10)

,topic,iteration,document,rel,text
0,00ecb565-8dcb-443e-875c-71babdce2269,0,638637447023194112,0,video shipper look to grab a piec of northwest...
1,00ecb565-8dcb-443e-875c-71babdce2269,0,638727826393444352,1,polic say woman wa drive drunk with kid in the...
2,00ecb565-8dcb-443e-875c-71babdce2269,0,639043756038074369,1,driver serious injur man in portola district a...
3,00ecb565-8dcb-443e-875c-71babdce2269,0,639219133859631104,0,updat man in critic condit after be shot in th...
4,00ecb565-8dcb-443e-875c-71babdce2269,0,639325701297041408,1,a barri woman that wa run over by a drunk driv...
5,00ecb565-8dcb-443e-875c-71babdce2269,0,639362932363653120,0,thi photo prove that there are sight in the no...
6,00ecb565-8dcb-443e-875c-71babdce2269,0,639485269385678849,1,a man wa hit by a car while tri to chase after...
7,00ecb565-8dcb-443e-875c-71babdce2269,0,639490286196932608,0,christopherokey the rule wa that he swung and ...
8,00ecb565-8dcb-443e-875c-71babdce2269,0,639535449673850880,0,my friend wa do cpr on him he took a coupl of ...
10,00ecb565-8dcb-443e-875c-71babdce2269,0,639662060368846848,1,rt the women who got out of her car to help ma...


In [61]:
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import gensim

doc1 = TaggedDocument(word_tokenize('My name is eugene'), 'tag1')
doc2 = TaggedDocument(word_tokenize('My name is tan'), 'tag2')
doc3 = TaggedDocument(word_tokenize('My name is alpha'), 'tag3')
sentences = [doc1, doc2, doc3]

model = Doc2Vec(alpha=0.025, min_alpha=0.025)
model.build_vocab(sentences)
# # for epoch in range(10):
model.train(sentences, total_examples=model.corpus_count, epochs=10)
# # model.save('/tmp/my_model.doc2vec')
# q = TaggedDocument('name is e', 'tag4')
# tokens = word_tokenize('name is e')
# query = model.infer_vector(tokens)
# # print(model.docvecs.most_similar(positive=[query], topn=10))
#     # model.alpha -= 0.002  # decrease the learning rate
#     # model.min_alpha = model.alpha
# print(model.dv)

RuntimeError: you must first build vocabulary before training the model

In [32]:
'''
CBOW tries to predict a word on the basis of its neighbors, while Skip Gram tries to predict the neighbors of a word.

In simpler words, CBOW tends to find the probability of a word occurring in a context. So, it generalizes over all the different contexts in which a word can be used. Whereas SkipGram tends to study different contexts separately. Skip Gram needs more data to be trained contains more knowledge about the context.
'''

from sklearn.feature_extraction.text import CountVectorizer
# model = Word2Vec(list(data.text), min_count=1,size= 50,workers=3, window =3, sg = 1)
vec = CountVectorizer()
term_doc = vec.fit_transform(list(data.text))
td_df = pd.DataFrame(term_doc.toarray(), columns=vec.get_feature_names(), index=data.document)

td_df.head()

,aaron,abad,abandon,abarrica,abba,abbey,abbeylynnmangu,abbott,abcgrandstand,abdel,...,zippydazipst,zone,zsofimajor,zuckerberg,zuma,zurich,zvishavan,zweigen,öbb,박근혜
document,,,,,,,,,,,,,,,,,,,,,
638637447023194112,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
638727826393444352,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
639043756038074369,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
639219133859631104,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
639325701297041408,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
from sklearn.metrics.pairwise import cosine_similarity

query = pd.DataFrame(vec.fit_transform(["hello my name is eugene"]))

cosine_similarity(td_df, query)

ValueError: setting an array element with a sequence.

In [ ]:
import gensim

